# Database Support

## Structure
* class Config
    * override \_\_init\_\_()
    * override save()
    * def get_related_log(self)
* class BaseLog
    * tmstamp

In [1]:
from IMP import NotebookLoader, NotebookFinder
from peewee import SqliteDatabase, Model, CharField, TextField, ForeignKeyField, DateTimeField, FloatField
from tools import get_addr_port, get_regs_list, get_reg_names, get_md5_string, cached_property, sort_dict
from IPython.core.debugger import Tracer; set_trace = Tracer(colors='linux')
from datetime import datetime
from time import time
from collections import OrderedDict


Docstring = '''
Database relative functional module
'''


db = SqliteDatabase('data.db')


# table config
class Config(Model):
    '''
    One config points to one log table. While insert a configure into config table,
    a related log table should be created. So in overrided function save(), there is 
    a get_related_log() to get that log table and create it.
    '''
    cached_md5 = {}
    sn = CharField(32)
    addr_info = CharField(32)
    regs_list = CharField(28)
    reg_names = CharField(128)
    info = TextField()
    md5 = CharField(32, unique=True)    
    class Meta:
        database = db

    def __init__(self, **argv):
        # md5 automatic generated
        if argv:
            argv = sort_dict(argv)
            md5string = ''.join(argv.values())
            md5 = argv['md5'] = get_md5_string(md5string)

            # data validation
            addr, self.port = get_addr_port(argv['addr_info'])
            rl = get_regs_list(argv['regs_list'])
            rn = get_reg_names(argv['reg_names'])        
            #print('All data valid')       

        super().__init__(**argv)  

    def save(self, force_insert=False, only=None):
        # when finished data validation, a related log table should be created(related on md5)
        # conventional saving process  
        super().save(force_insert, only)
        #print('saved')        

        # create new Log Table class and create it in database
        Log = self.get_related_log()
        Log.create_table()


    def get_related_log(self):
        assert(self is not None)
        if self.md5 in Config.cached_md5:
            return Config.cached_md5[self.md5]
        ref_dict = OrderedDict()
        rl = get_regs_list(self.regs_list)
        rn = get_reg_names(self.reg_names)
        for name, reg in zip(rn, rl):
            ref_dict['_'.join([name, str(reg)])] = FloatField()

        ref_dict['__module__'] = getattr(self, '__module__', None)

        LogTable = type(self.md5, (BaseLog,), ref_dict)
        Config.cached_md5.update({self.md5: LogTable})
        return LogTable


class BaseLog(Model):            
    tmstamp = DateTimeField()
    class Meta:
        database = db

importing Jupyter notebook from tools.ipynb


In [27]:
import numpy as np
import pandas as pd


def show_all_config():
    s = Config.select()
    data_list = [d._data for d in s]
    df = pd.DataFrame(data_list)
    df.index = df.id
    del df['id']
    return df

In [29]:
show_all_config()

,addr_info,info,md5,reg_names,regs_list,sn
id,,,,,,
1,192.168.1.10@202,info: for testing,5821d5682b9b876d18aa8cb3cc67b88f,"ch1,ch2,ch3,ch4,ch5","20-23,50",slv0309
2,192.168.1.10@202,info: for testing,f766c6ec2a01e03f0f5d981d711091dd,"ch1,ch2,ch3,ch4,ch5","20-23,50",slv1204
3,192.168.1.10@202,info: for testing,952984c1ce79f919a01cbb3eb842e945,"ch1,ch2,ch3,ch4,ch5","20-23,50",slv1021
